*  This program compares the performance 
*                             of the 4 investement rules/strategies 
*                     (not including the correlation shrinkage)
*  when applies to the 5 industry portfolios      
*  We impose further portfolio constraints:  a < = w_j <= b 

In [1]:
import pandas as pd                     # To load data, we use the package pandas
import numpy as np
import scipy.special as sc                # to compute the beta function 


# Load the data 

df = pd.read_excel('Factors_July26_July11.xlsx')        # It has 5 columns:  date, mkt, size, b/m, riskree rate
                                                    # downloaded from Ken French's website
df2 = pd.read_excel('Indu5_July26_July11.xlsx')       # return on 5 industry portfolios from Ken French's web 

mkt = df.loc[:,"mkt"]/100                       # Mkt excess return;  divided by 100 due to data are in %
rf = df.loc[:,"rate"]/100

R1 = df2.loc[:,'Indu1' : 'Indu5']/100        # extract the 5 industry returns, R1 is T by 5. 

R1 = np.array(R1)                         # convert list to array to apply np.functions
rf = np.array(rf)                         # convert list to array to apply np.functions
T = len(df)                               # The number of obs
Re = np.ones((T,5))                           # creat storage for excess returns

for i in range(5):
    Re[:,i] = R1[:,i] - rf                # the excess return:  each indu substracts riskfree rate, Re[:,i]-rf 

T0 = 120                 # set aside data for the initial estimation

print(' Total sample size \n')
print(T)
print(' Sample size used to start estimation \n')
print(T0)

 Total sample size 

1021
 Sample size used to start estimation 

120


In [2]:
# Compute the first three portfolio weights, which are simpler
 

TT = T - T0              #  from T0, T0+1, ..., to T-1
                         #  No need for T as we will not know its return

                         # declare storage for the 4 portfolios (though not necessary, but clearer)
                                      #          each is 5-vector over time
w1 = np.ones((5,TT))
w2 = np.ones((5,TT))
w3 = np.ones((5,TT))
w4 = np.ones((5,TT))

# write a function to get the constrained portfolio weights:

def get_Weights(gamma,mu,V):

    import cvxopt        # install it first: "pip install cvxopt" via Spyder
    from cvxopt import matrix, solvers
    Q = gamma * V        # the mapping of utility paramters into quadratic programming
    Q = matrix(Q)
    q = -mu
    q = matrix(q)
    
    lb = np.ones((1, 5)) * 0.0                          # lower bound
    ub = np.ones((1, 5)) * 0.4                         # upper bound
    h = np.append(ub, lb, axis = 1)
    h = h.T
    h = matrix(h)
 
    G1 = np.eye(5)                           # identiy matrix of order 5
    G = np.append(G1, -G1, axis = 0)
    G = G
    G = matrix(G)
 
    sol = solvers.qp(Q, q, G, h)     # Format:  solvers.qp(Q, q, G, h, A, b) 
    sola = np.array(sol['x'])
    return sola.T

                    # 1):  1/N rule over T0, T0+1, ..., T-1
N = 5
w1 = np.ones((5,TT)) / N       

                    # 2):  The other 3 rules

gamma = 3     
ones5 = np.ones((5,1))            # a 5-vector of ones, to be used later

for t in range(T0,T):
    RR = np.ones((t,5))           # to store the data from 1 to t, t by 5 matrix 
    for i in range (t):           # get available data up to t
        RR[i,:] = Re[i,:] 
    mu = np.mean(RR, axis = 0)          # the mean taking each column of the matrix 
    mu = mu.T                           # make it a column vector
    V = np.cov(RR.T)                        # the covariance estimate 
    VI = np.linalg.inv(V)                     #  Inverse of V
    tt = t-T0
    w2[:,tt] = get_Weights(gamma,mu,V)       # store the plug-in weights
    A = np.matmul(VI, ones5)
    b = np.matmul(ones5.T, A)
    w3C = A / b                                  #  5 by 1, GMV weights
    for j in range(5):
        w3[j,tt] = w3C[j]                      # store to w3
                              # Now the last and complex weights 
    B = np.matmul(VI, mu) 
    theta2 = np.matmul(mu.T, B)             # sample Sharpe ratio squared
    x = theta2 / (1+ theta2)                # get parameters for incomplete beta function
    a = N/2.0
    b = (t-N) / 2.0
    ibeta = sc.betainc(a, b,x)*sc.beta(a, b)   # b/c sc.betainc defined with 1/sc.beta(a, b) coeff
    c = ((t-N-2)*theta2 - N)/t
    d = 2*np.power(theta2,N/2.0)*np.power(1+theta2,-(t-2)/2.0)
    theta2a = c + d / (t*ibeta)
    c1 = (t-2)*(t-N-2) / ( (t-N-1)*(t-N-4) )
    A = np.matmul(V, ones5)                 # variables in pi1 and pi2
    B = np.matmul(ones5.T, A) 
    C = np.matmul(ones5.T, mu) / N
    pi1 = B / (N*N) - (2*C)/gamma + theta2a / (gamma*gamma)
    pi2 = (c1-1)*theta2a / (gamma*gamma) + c1*N / (t*gamma*gamma)
    if pi1 < 0:                            # sets pi1 to be 0 or positive
        pi1 = 0
    delta = pi1 / (pi1+pi2)
    w2C=((t-N-2)/t)*w2[:,tt]                # the adjusted plug-in rule
    w2C.shape = (5,1)                       # make sure the next Equ gets 5 by 1 vector
    w4C = (1-delta)*ones5/N +  delta*w2C     # the combination weights 
                    # (t-N-2)/t is due to using the adjusted plug-in
    for j in range(5):
        w4[j,tt] = w4C[j]                      # store to w4

print('The first weights at T0 \n')
print(w1[:,0])
print('The weights \n')
print(w2[:,0])
print('The weights \n')
print(w3[:,0])
print('The weights \n')
print(w4[:,0])

     pcost       dcost       gap    pres   dres
 0:  4.8598e-03 -2.0204e+00  2e+00  0e+00  3e-16
 1:  4.5340e-03 -3.8743e-02  4e-02  8e-17  4e-16
 2: -2.5312e-04 -5.1348e-03  5e-03  1e-16  2e-17
 3: -1.4648e-03 -2.1463e-03  7e-04  1e-16  5e-18
 4: -1.8913e-03 -2.0415e-03  2e-04  2e-16  4e-18
 5: -1.9258e-03 -1.9355e-03  1e-05  2e-16  4e-18
 6: -1.9314e-03 -1.9321e-03  7e-07  2e-16  4e-18
 7: -1.9317e-03 -1.9319e-03  2e-07  2e-16  4e-18
 8: -1.9319e-03 -1.9319e-03  2e-08  2e-16  4e-18
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  4.3695e-03 -2.0201e+00  2e+00  0e+00  2e-16
 1:  4.0636e-03 -3.8541e-02  4e-02  1e-16  2e-16
 2: -4.7807e-04 -5.2520e-03  5e-03  9e-17  2e-17
 3: -1.6181e-03 -2.3664e-03  7e-04  6e-17  4e-18
 4: -2.0908e-03 -2.2563e-03  2e-04  1e-16  2e-18
 5: -2.1307e-03 -2.1498e-03  2e-05  8e-17  5e-18
 6: -2.1320e-03 -2.1382e-03  6e-06  2e-16  5e-18
 7: -2.1352e-03 -2.1355e-03  3e-07  1e-16  4e-18
 8: -2.1353e-03 -2.1353e-03  4e-09  2e-16  4e-1

 3: -9.1736e-04 -1.4616e-03  5e-04  2e-16  2e-18
 4: -1.2780e-03 -1.3712e-03  9e-05  2e-16  3e-18
 5: -1.2941e-03 -1.2956e-03  2e-06  2e-16  3e-18
 6: -1.2944e-03 -1.2944e-03  2e-08  2e-16  3e-18
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  5.4578e-03 -2.0194e+00  2e+00  0e+00  2e-16
 1:  5.1509e-03 -3.7817e-02  4e-02  1e-16  3e-16
 2:  3.2848e-04 -4.5855e-03  5e-03  2e-16  3e-17
 3: -1.0248e-03 -1.5789e-03  6e-04  1e-16  3e-18
 4: -1.3972e-03 -1.5022e-03  1e-04  2e-16  2e-18
 5: -1.4181e-03 -1.4202e-03  2e-06  1e-16  3e-18
 6: -1.4186e-03 -1.4186e-03  2e-08  7e-17  3e-18
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  5.2991e-03 -2.0193e+00  2e+00  0e+00  2e-16
 1:  5.0001e-03 -3.7706e-02  4e-02  1e-16  4e-16
 2:  2.6951e-04 -4.5876e-03  5e-03  1e-16  2e-17
 3: -1.0495e-03 -1.6417e-03  6e-04  8e-17  4e-18
 4: -1.4418e-03 -1.5643e-03  1e-04  2e-16  2e-18
 5: -1.4668e-03 -1.4709e-03  4e-06  9e-17  1e-18
 6: -1.4678e-03 -1.4679

 6: -1.8532e-03 -1.8537e-03  5e-07  3e-16  2e-18
 7: -1.8535e-03 -1.8536e-03  5e-08  1e-16  3e-18
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  2.4201e-03 -2.0146e+00  2e+00  0e+00  2e-16
 1:  2.2721e-03 -3.3822e-02  4e-02  2e-16  3e-16
 2: -6.1187e-04 -4.6642e-03  4e-03  1e-16  2e-17
 3: -1.5085e-03 -2.0349e-03  5e-04  1e-16  2e-18
 4: -1.8220e-03 -1.9264e-03  1e-04  8e-17  3e-18
 5: -1.8414e-03 -1.8464e-03  5e-06  2e-16  3e-18
 6: -1.8432e-03 -1.8436e-03  5e-07  2e-16  1e-18
 7: -1.8434e-03 -1.8435e-03  5e-08  3e-16  2e-18
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  2.3910e-03 -2.0145e+00  2e+00  0e+00  2e-16
 1:  2.2447e-03 -3.3760e-02  4e-02  1e-16  2e-16
 2: -6.1701e-04 -4.6581e-03  4e-03  1e-16  1e-17
 3: -1.5077e-03 -2.0439e-03  5e-04  8e-17  4e-18
 4: -1.8254e-03 -1.9357e-03  1e-04  1e-16  1e-18
 5: -1.8455e-03 -1.8510e-03  5e-06  1e-16  2e-18
 6: -1.8470e-03 -1.8475e-03  4e-07  1e-16  3e-18
 7: -1.8471e-03 -1.8472

 5: -2.0895e-03 -2.0896e-03  1e-07  1e-16  2e-18
 6: -2.0895e-03 -2.0895e-03  1e-09  1e-16  2e-18
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  8.6707e-04 -2.0128e+00  2e+00  0e+00  4e-16
 1:  7.7777e-04 -3.2228e-02  3e-02  1e-16  4e-16
 2: -1.1933e-03 -4.9095e-03  4e-03  9e-17  3e-17
 3: -1.8757e-03 -2.1552e-03  3e-04  1e-16  4e-18
 4: -2.0507e-03 -2.0655e-03  1e-05  2e-16  2e-18
 5: -2.0562e-03 -2.0563e-03  2e-07  2e-16  2e-18
 6: -2.0562e-03 -2.0562e-03  2e-09  1e-16  3e-18
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  1.0500e-03 -2.0127e+00  2e+00  0e+00  1e-16
 1:  9.5765e-04 -3.2193e-02  3e-02  5e-17  4e-16
 2: -1.0776e-03 -4.8113e-03  4e-03  1e-16  4e-17
 3: -1.7789e-03 -2.0715e-03  3e-04  5e-17  3e-18
 4: -1.9543e-03 -1.9695e-03  2e-05  1e-16  2e-18
 5: -1.9603e-03 -1.9604e-03  2e-07  1e-16  3e-18
 6: -1.9603e-03 -1.9603e-03  2e-09  2e-16  2e-18
Optimal solution found.
     pcost       dcost       gap    pres   dres


 4: -2.5774e-03 -2.6219e-03  4e-05  2e-16  3e-18
 5: -2.5937e-03 -2.5966e-03  3e-06  2e-16  3e-18
 6: -2.5955e-03 -2.5955e-03  5e-08  2e-16  1e-18
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -7.0686e-04 -2.0112e+00  2e+00  0e+00  2e-16
 1: -7.5469e-04 -3.0866e-02  3e-02  9e-17  5e-16
 2: -1.9607e-03 -5.4080e-03  3e-03  7e-17  4e-17
 3: -2.4561e-03 -2.8605e-03  4e-04  1e-16  1e-18
 4: -2.5962e-03 -2.6378e-03  4e-05  2e-16  1e-18
 5: -2.6142e-03 -2.6166e-03  2e-06  1e-16  2e-18
 6: -2.6158e-03 -2.6159e-03  5e-08  2e-16  2e-18
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -7.0091e-04 -2.0111e+00  2e+00  0e+00  1e-16
 1: -7.4837e-04 -3.0800e-02  3e-02  1e-16  4e-16
 2: -1.9468e-03 -5.3841e-03  3e-03  1e-16  5e-17
 3: -2.4390e-03 -2.7710e-03  3e-04  2e-16  2e-18
 4: -2.5621e-03 -2.6022e-03  4e-05  1e-16  2e-18
 5: -2.5820e-03 -2.5845e-03  3e-06  5e-17  2e-18
 6: -2.5837e-03 -2.5838e-03  5e-08  1e-16  9e-19
Optimal solution found.

...

 5: -3.1466e-03 -3.1498e-03  3e-06  2e-16  9e-19
 6: -3.1480e-03 -3.1482e-03  2e-07  9e-17  2e-18
 7: -3.1480e-03 -3.1480e-03  3e-09  2e-16  1e-18
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -2.3547e-03 -2.0066e+00  2e+00  0e+00  9e-17
 1: -2.3608e-03 -2.6573e-02  2e-02  9e-17  2e-16
 2: -2.6176e-03 -5.0160e-03  2e-03  1e-16  2e-17
 3: -2.9660e-03 -3.3519e-03  4e-04  1e-16  3e-18
 4: -3.1254e-03 -3.1716e-03  5e-05  1e-16  1e-18
 5: -3.1516e-03 -3.1545e-03  3e-06  1e-16  2e-18
 6: -3.1528e-03 -3.1529e-03  1e-07  2e-16  1e-18
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -2.3075e-03 -2.0067e+00  2e+00  0e+00  2e-16
 1: -2.3138e-03 -2.6594e-02  2e-02  8e-17  3e-16
 2: -2.5800e-03 -5.0034e-03  2e-03  1e-16  3e-17
 3: -2.9387e-03 -3.3335e-03  4e-04  9e-17  3e-18
 4: -3.1023e-03 -3.1514e-03  5e-05  2e-16  1e-18
 5: -3.1306e-03 -3.1337e-03  3e-06  2e-16  2e-18
 6: -3.1318e-03 -3.1319e-03  1e-07  1e-16  2e-18
 7: -3.1318e-03 -3.1318

 5: -3.7181e-03 -3.7186e-03  5e-07  2e-16  2e-18
 6: -3.7183e-03 -3.7183e-03  5e-09  1e-16  1e-18
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -3.0562e-03 -2.0063e+00  2e+00  0e+00  3e-16
 1: -3.0593e-03 -2.6252e-02  2e-02  1e-16  4e-16
 2: -3.2173e-03 -5.2800e-03  2e-03  7e-17  4e-17
 3: -3.6245e-03 -3.8541e-03  2e-04  1e-16  4e-18
 4: -3.7584e-03 -3.7738e-03  2e-05  2e-16  2e-18
 5: -3.7658e-03 -3.7661e-03  4e-07  2e-16  2e-18
 6: -3.7659e-03 -3.7659e-03  4e-09  1e-16  1e-18
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -3.0243e-03 -2.0063e+00  2e+00  0e+00  2e-16
 1: -3.0275e-03 -2.6243e-02  2e-02  1e-16  3e-16
 2: -3.1897e-03 -5.2577e-03  2e-03  1e-16  2e-17
 3: -3.6066e-03 -3.8307e-03  2e-04  2e-16  3e-18
 4: -3.7416e-03 -3.7542e-03  1e-05  2e-16  2e-18
 5: -3.7475e-03 -3.7477e-03  2e-07  1e-16  2e-18
 6: -3.7476e-03 -3.7476e-03  2e-09  2e-16  3e-18
Optimal solution found.
     pcost       dcost       gap    pres   dres


     pcost       dcost       gap    pres   dres
 0: -2.7423e-03 -2.0063e+00  2e+00  0e+00  1e-16
 1: -2.7462e-03 -2.6216e-02  2e-02  1e-16  4e-16
 2: -2.9351e-03 -5.0898e-03  2e-03  7e-17  4e-17
 3: -3.3443e-03 -3.6123e-03  3e-04  1e-16  2e-18
 4: -3.4877e-03 -3.5204e-03  3e-05  2e-16  2e-18
 5: -3.5012e-03 -3.5030e-03  2e-06  1e-16  1e-18
 6: -3.5022e-03 -3.5024e-03  1e-07  1e-16  7e-19
 7: -3.5023e-03 -3.5023e-03  1e-09  2e-16  1e-18
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -2.7225e-03 -2.0062e+00  2e+00  0e+00  2e-16
 1: -2.7265e-03 -2.6198e-02  2e-02  1e-16  4e-16
 2: -2.9162e-03 -5.0679e-03  2e-03  8e-17  4e-17
 3: -3.3220e-03 -3.5947e-03  3e-04  1e-16  3e-18
 4: -3.4687e-03 -3.5023e-03  3e-05  1e-16  8e-19
 5: -3.4828e-03 -3.4847e-03  2e-06  1e-16  1e-18
 6: -3.4838e-03 -3.4839e-03  1e-07  1e-16  1e-18
 7: -3.4839e-03 -3.4839e-03  1e-09  9e-17  1e-18
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -2.6281e-03 -2.0062e

     pcost       dcost       gap    pres   dres
 0: -2.7085e-03 -2.0060e+00  2e+00  0e+00  1e-16
 1: -2.7120e-03 -2.5949e-02  2e-02  2e-16  6e-16
 2: -2.8788e-03 -4.9251e-03  2e-03  7e-17  5e-17
 3: -3.2264e-03 -3.4959e-03  3e-04  1e-16  3e-18
 4: -3.3501e-03 -3.3746e-03  2e-05  1e-16  1e-18
 5: -3.3597e-03 -3.3615e-03  2e-06  2e-16  3e-18
 6: -3.3603e-03 -3.3605e-03  2e-07  2e-16  1e-18
 7: -3.3603e-03 -3.3604e-03  2e-08  1e-16  2e-18
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -2.7385e-03 -2.0060e+00  2e+00  0e+00  3e-16
 1: -2.7419e-03 -2.5932e-02  2e-02  1e-16  6e-16
 2: -2.9035e-03 -4.9315e-03  2e-03  8e-17  5e-17
 3: -3.2446e-03 -3.5095e-03  3e-04  7e-17  4e-18
 4: -3.3650e-03 -3.3890e-03  2e-05  2e-16  6e-19
 5: -3.3740e-03 -3.3757e-03  2e-06  1e-16  1e-18
 6: -3.3744e-03 -3.3746e-03  2e-07  1e-16  2e-18
 7: -3.3744e-03 -3.3744e-03  7e-09  2e-16  7e-19
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -2.7354e-03 -2.0060e

Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -2.1215e-03 -2.0064e+00  2e+00  0e+00  2e-16
 1: -2.1274e-03 -2.6335e-02  2e-02  7e-17  1e-16
 2: -2.3912e-03 -4.8441e-03  2e-03  9e-17  1e-17
 3: -2.7944e-03 -3.1839e-03  4e-04  1e-16  1e-18
 4: -2.9473e-03 -2.9951e-03  5e-05  2e-16  1e-18
 5: -2.9724e-03 -2.9751e-03  3e-06  2e-16  8e-19
 6: -2.9739e-03 -2.9740e-03  6e-08  1e-16  1e-18
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -2.1992e-03 -2.0064e+00  2e+00  0e+00  8e-17
 1: -2.2048e-03 -2.6292e-02  2e-02  1e-16  3e-16
 2: -2.4524e-03 -4.8559e-03  2e-03  1e-16  3e-17
 3: -2.8338e-03 -3.2101e-03  4e-04  1e-16  4e-18
 4: -2.9809e-03 -3.0252e-03  4e-05  2e-16  2e-18
 5: -3.0038e-03 -3.0064e-03  3e-06  1e-16  2e-18
 6: -3.0053e-03 -3.0054e-03  8e-08  1e-16  1e-18
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -2.2449e-03 -2.0063e+00  2e+00  0e+00  4e-16
 1: -2.2503e-03 -2.6273e-02  2e-02  9e-17  5e-16


In [3]:
# Compute the Sharpe Ratios and Accu Return of the 4 rules and Mkt
        
       # write a function to do it to save space or to be cleaer

def get_SRW(w):
    Port=np.ones((TT,1))      #   to store the returns on the portfolio
    ExPort = np.ones((TT,1))      # to store excess return
    for t in range(TT):
        w5 = w[:,t]
        ExPort[t] = np.dot(w5,Re[T0+t])      # excess return of the portfolio
        Port[t] = ExPort[t] + rf[T0+t]       # add back r_f to get the raw return
    muP = ExPort.mean()
    sig2P = ExPort.var()
    sigP = np.sqrt(sig2P)
    SharpeP = np.sqrt(12)*muP/sigP        # annualize the Sharp Ratio
    CC_Port = np.ones((TT,1))          # to store the accumulative returns 
    CC_Port[0] = 1+Port[0]  
    for t in range(TT-1):
        CC_Port[t+1] = CC_Port[t]*(1+Port[t+1])
    c = CC_Port[TT-1]
    return SharpeP, c

sr1, c1 = get_SRW(w1)
sr2, c2 = get_SRW(w2)
sr3, c3 = get_SRW(w3)
sr4, c4 = get_SRW(w4)

# For the mkt
                                     # Sharpe ratio of the Market 
                                     #  over  T0+1, ..., T 
mkt1 = df.loc[T0:T,"mkt"]/100       # Note r_{T0+1} is stored as r[T0]
mu = mkt1.mean()                      
sig2 = mkt1.var()                       
sigma = np.sqrt(sig2)                  
SharpeM = np.sqrt(12)*mu/sigma      # annualized      

# Get the accu return for the mkt

CC = np.ones((TT,1))          # to store the accumulative returns 
                           # raw return by adding rf back
mkt2 = (df.loc[T0:T,"mkt"] + df.loc[T0:T,"rate"])/100  

print('The Mkt raw returns \n')
print (mkt2.head()) 

mkt2 = np.array(mkt2)       # the index will start from 0

CC[0] = 1 + mkt2[0]                 # initial accu return

for t in range(TT-1):
    CC[t+1] = CC[t]*(1+mkt2[t+1])
    
print('The Annualized Sharpe ratios of the Opt Port and Mkt \n')
print('          {0:.4f}   {1:.4f}  \n'.format(sr1, SharpeM)) 
print('          {0:.4f}   {1:.4f}  \n'.format(sr2, SharpeM)) 
print('          {0:.4f}   {1:.4f}  \n'.format(sr3, SharpeM)) 
print('          {0:.4f}   {1:.4f}  \n'.format(sr4, SharpeM)) 

print('Terminal wealth of the 4 rules  \n')
print(c1)  
print(c2)  
print(c3)  
print(c4) 
print('Terminal wealth in for investing in Mkt  \n')
print(CC[TT-1])  


The Mkt raw returns 

120    0.0646
121    0.0108
122    0.0143
123    0.0691
124    0.0333
dtype: float64
The Annualized Sharpe ratios of the Opt Port and Mkt 

          0.5127   0.4610  

          0.4767   0.4610  

          0.5129   0.4610  

          0.5068   0.4610  

Terminal wealth of the 4 rules  

[2568.30219166]
[664.67795996]
[2529.46767975]
[1441.70287544]
Terminal wealth in for investing in Mkt  

[1565.54242462]


In [ ]:
prin